# Format signal component results

Pandas code to produce the tables in the appendix of `MCC`.

After formatting, tables are fed into [Tables Generator](https://www.tablesgenerator.com/latex_tables) to produce final latex version.

In [1]:
import pandas as pd
import numpy as np
from graspologic.datasets import load_mice

pd.options.mode.chained_assignment = None

In [2]:
mice = load_mice()
mice.atlas.head()

,ROI,Structure,Abbreviation,c_R,c_G,c_B,c_A,Level_1,Level_2,Level_3,Level_4,Macrostructure
0,1,Cingulate_Cortex_Area_24a,A24a,255,0,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
1,2,Cingulate_Cortex_Area_24a_prime,A24aPrime,255,124,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
2,3,Cingulate_Cortex_Area_24b,A24b,255,248,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
3,4,Cingulate_Cortex_Area_24b_prime,A24bPrime,138,255,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex
4,5,Cingulate_Cortex_Area_29a,A29a,14,255,0,255,forebrain,secondary_prosencephalon,isocortex,cingulate_cortex,isocortex


In [3]:
ventricles = [147, 151, 160]
ventricles += [roi + 166 for roi in ventricles]
ventricles = np.array(ventricles)

def lookup_roi_name(roi):
    roi += np.sum(roi >= ventricles) # Adjust for removing the ventricles
    roi -= 1
    hemisphere = "R" if roi // 166 else "L"
    roi = roi % 166
    structure = mice.atlas["Structure"].values[roi]
    structure = " ".join(structure.split("_"))
    return f"{structure} ({hemisphere})"

## Table 1: Signal Edges

The top 20 signal edges (out of 54,946 total edges) ranked by the order of their Holm--Bonferroni corrected $p$-value. Eleven of the top 20 signal edges are adjacent to either the left or right hemisphere corpus callosum.

In [4]:
signal_edges = pd.read_csv("../results/signal_edges.csv")

# Get the top 20 strongest signal edges
strong_signal_edges = signal_edges.head(20)
strong_signal_edges["ROI_1"] = strong_signal_edges["ROI_1"].apply(lookup_roi_name)
strong_signal_edges["ROI_2"] = strong_signal_edges["ROI_2"].apply(lookup_roi_name)

strong_signal_edges.drop(["pvalue", "significant"], axis=1, inplace=True)
strong_signal_edges.columns = ["Vertex 1", "Vertex 2", "statistic", "p-value"]

strong_signal_edges["statistic"] = strong_signal_edges["statistic"].apply(lambda x: f"{x:.3f}")
strong_signal_edges["p-value"] = strong_signal_edges["p-value"].apply(lambda x: f"{x:.3f}")

strong_signal_edges

,Vertex 1,Vertex 2,statistic,p-value
0,Corpus Callosum (L),Striatum (R),0.717,0.053
1,Corpus Callosum (L),Internal Capsule (R),0.699,0.070
2,Corpus Callosum (L),Reticular Nucleus of Thalamus (R),0.698,0.072
3,Corpus Callosum (L),Zona Incerta (R),0.686,0.088
4,Septum (R),Corpus Callosum (R),0.671,0.113
5,Striatum (L),Striatum (R),0.664,0.127
6,Corpus Callosum (L),Ventral Thalamic Nuclei (R),0.663,0.128
7,Hippocampus (L),Middle Cerebellar Peduncle (L),0.658,0.139
8,Caudomedial Entorhinal Cortex (R),Ventral Hippocampal Commissure (R),0.656,0.145
9,Corpus Callosum (L),Midbrain Reticular Nucleus (R),0.653,0.154


In [5]:
strong_signal_edges.round(3).to_csv("../results/formatted_tables/edges.csv", index=False)

## Table 2: Signal Vertices

The top 20 signal vertices (out of 332 total vertices) ranked by the order of their Holm--Bonferroni corrected $p$-values. Pillai's trace and approximate $F$ statistic (along with degrees of freedom) as calculated by one-way MANOVA are also reported. The corpus callosum in the left and right hemisphere are the top two signal vertices.

In [6]:
signal_vertices = pd.read_csv("../results/signal_vertices.csv")

# Get the top 10 strongest signal edges
strong_signal_vertices = signal_vertices.head(20)
strong_signal_vertices["ROI"] = strong_signal_vertices["ROI"].apply(lookup_roi_name)
strong_signal_vertices

strong_signal_vertices.drop(["num.df", "den.df", "pvalue", "significant"], axis=1, inplace=True)
strong_signal_vertices.columns = ["Vertex", "Pillai", "F(15, 78)", "p-value"]

# pd.set_option('display.float_format', '{:.3g}'.format)
strong_signal_vertices["Pillai"] = strong_signal_vertices["Pillai"].apply(lambda x: f"{x:.3f}")
strong_signal_vertices["F(15, 78)"] = strong_signal_vertices["F(15, 78)"].apply(lambda x: f"{x:.3f}")
strong_signal_vertices["p-value"] = strong_signal_vertices["p-value"].apply(lambda x: f"{x:.3g}")
strong_signal_vertices

,Vertex,Pillai,"F(15, 78)",p-value
0,Corpus Callosum (L),2.592,33.026,4.47e-25
1,Corpus Callosum (R),2.550,29.501,1.75e-23
2,Secondary Motor Cortex (L),2.438,22.578,7.73e-20
3,Midbrain Reticular Nucleus (R),2.435,22.429,9.43e-20
4,Fimbria (L),2.424,21.902,1.94e-19
5,Substantia Nigra (R),2.299,17.049,3.04e-16
6,Secondary Motor Cortex (R),2.298,17.034,3.1e-16
7,Internal Capsule (R),2.297,17.003,3.26e-16
8,Striatum (L),2.259,15.854,2.32e-15
9,Cerebral Peduncle (R),2.229,15.036,9.98e-15


In [7]:
strong_signal_vertices.to_csv("../results/formatted_tables/vertices.csv", index=False)

## Table 3: Bilateral Signal Pairs

The top 10 bilateral signal vertex pairs (out of 166 total vertex pairs) ranked by the order of their average total rank.

In [8]:
def roi_name_no_hemisphere(roi):
    name = lookup_roi_name(roi)
    name = " ".join(name.split(" ")[:-1])
    return name

In [9]:
def bilateral_twin(roi):
    left = True
    if roi > 163:
        left = False
        return roi - 163, left
    else:
        return roi + 163, left

In [10]:
def bilateral_ranks(roi):

    bilateral_roi, left = bilateral_twin(roi)
    if left:
        left_roi = roi
        right_roi = bilateral_roi
    else:
        left_roi = bilateral_roi
        right_roi = roi
        
    left_rank = bilateral.query(f"ROI == {left_roi}")["rank"].values[0]
    right_rank = bilateral.query(f"ROI == {right_roi}")["rank"].values[0]

    left_pval = bilateral.query(f"ROI == {left_roi}")["holm_pvalue"].values[0]
    right_pval = bilateral.query(f"ROI == {right_roi}")["holm_pvalue"].values[0]


    return left_pval, left_rank, right_pval, right_rank

In [11]:
def avg_rank(row):
    return (row.left_rank + row.right_rank) / 2

In [12]:
bilateral = signal_vertices.copy()
bilateral = bilateral.drop(["pillai", "F", "num.df", "den.df", "pvalue", "significant"], axis=1)
bilateral["rank"] = bilateral["holm_pvalue"].rank().astype(int)
bilateral["name"] = bilateral["ROI"].apply(roi_name_no_hemisphere)

bilateral["left_pval"], bilateral["left_rank"], bilateral["right_pval"], bilateral["right_rank"] = zip(*bilateral["ROI"].apply(bilateral_ranks))
bilateral = bilateral.drop(["ROI", "holm_pvalue", "rank"], axis=1)
bilateral = bilateral.drop_duplicates()

bilateral["left_pval"] = bilateral["left_pval"].apply(lambda x: f"{x:.3g}")
bilateral["right_pval"] = bilateral["right_pval"].apply(lambda x: f"{x:.3g}")

bilateral["avg_rank"] = bilateral.apply(avg_rank, axis=1)
bilateral = bilateral.sort_values(["avg_rank"])

bilateral.head(10)

,name,left_pval,left_rank,right_pval,right_rank,avg_rank
0,Corpus Callosum,4.47e-25,1,1.75e-23,2,1.5
2,Secondary Motor Cortex,7.73e-20,3,3.1e-16,7,5.0
7,Internal Capsule,1.6e-14,11,3.26e-16,8,9.5
11,Stria Terminalis,1.25e-13,18,2.59e-14,12,15.0
4,Fimbria,1.94e-19,5,3.58e-12,26,15.5
20,Ventral Tegmental Area,3.87e-12,27,8.76e-13,21,24.0
14,Hippocampus,8.02e-11,40,7.43e-14,15,27.5
28,Ectorhinal Cortex,8.11e-12,29,4.56e-11,37,33.0
24,Globus Pallidus,9.68e-11,41,1.89e-12,25,33.0
9,Cerebral Peduncle,4.64e-10,57,9.98e-15,10,33.5


In [13]:
bilateral.head(10).to_csv("../results/formatted_tables/vertices_bilateral.csv", index=False)

## Table 4: Signal Communities

The top 10 signal communities (out of 105 total communities) ranked by the order of their Holm--Bonferroni corrected $p$-values as calculated by the Multivariate Weighted method.

In [14]:
signal_communities = pd.read_csv("../results/signal_communities.csv")

signal_communities = signal_communities.drop(["pvalue", "significant"], axis=1)

signal_communities["statistic"] = signal_communities["statistic"].apply(lambda x: f"{x:.3f}")
signal_communities["holm_pvalue"] = signal_communities["holm_pvalue"].apply(lambda x: f"{x:.3g}")

signal_communities.head()

,Community 1,Community 2,statistic,holm_pvalue
0,White Matter (R),White Matter (R),0.885,6.43e-06
1,White Matter (L),White Matter (L),0.857,1.02e-05
2,Hindbrain (L),White Matter (L),0.849,1.14e-05
3,Midbrain (R),White Matter (R),0.845,1.21e-05
4,Isocortex (L),Isocortex (L),0.844,1.22e-05


In [15]:
signal_communities.head(10).to_csv("../results/formatted_tables/communities.csv", index=False)